<a href="https://colab.research.google.com/github/pcamarillor/O2024_ESI3914O/blob/Lab13_Team08/examples/notebooks/kmeans/lab13_solution_team08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# Check this site for the latest download link
# https://www.apache.org/dyn/closer.lua/spark
!wget -q https://dlcdn.apache.org/spark/spark-3.5.2/spark-3.5.2-bin-hadoop3.tgz
!tar xf spark-3.5.2-bin-hadoop3.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,700 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,466 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages 

In [8]:
from pyspark.sql import SparkSession
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.feature import VectorAssembler

# Initialize Spark session
spark = SparkSession.builder.appName("WineClustering").getOrCreate()

# Load the CSV file
file_path = '/content/drive/My Drive/wine-clustering.csv'
wine_df = spark.read.csv(file_path, header=True, inferSchema=True)

# Assemble features into a single vector column
feature_columns = [col for col in wine_df.columns if col != "label"]
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
wine_data = assembler.transform(wine_df).select("features")

# Define K values to evaluate
k_values = [2, 10, 15, 20]
silhouette_scores = {}

for k in k_values:
    # Train the K-means model
    kmeans = KMeans().setK(k).setSeed(1).setFeaturesCol("features")
    model = kmeans.fit(wine_data)

    # Make predictions
    predictions = model.transform(wine_data)

    # Evaluate the model with silhouette score
    evaluator = ClusteringEvaluator(featuresCol="features", metricName="silhouette")
    silhouette = evaluator.evaluate(predictions)
    silhouette_scores[k] = silhouette
    print(f"Silhouette score for k={k}: {silhouette}")

# Display silhouette scores
print("Silhouette Scores:", silhouette_scores)

# Stop the Spark session
spark.stop()


Silhouette score for k=2: 0.8213603513331723
Silhouette score for k=10: 0.6810238482437947
Silhouette score for k=15: 0.6844672306874956
Silhouette score for k=20: 0.6393344257672275
Silhouette Scores: {2: 0.8213603513331723, 10: 0.6810238482437947, 15: 0.6844672306874956, 20: 0.6393344257672275}
